In [37]:
from ctypes import *
import os
import time
import pandas as pd

In [15]:
# Get the current working directory
current_dir = os.getcwd()

# Define relative paths for the DLLs
dll_relative_path = "../python-c-benchmark/scripts/func64.dll"
dll_relative_path_o3 = "../python-c-benchmark/scripts/func64_o3.dll"

# Construct absolute paths for the DLLs
absolute_path = os.path.abspath(os.path.join(current_dir, dll_relative_path))
absolute_path_o3 = os.path.abspath(os.path.join(current_dir, dll_relative_path_o3))

# Load both DLLs
my_func = WinDLL(absolute_path, winmode=0x8)
my_func_o3 = WinDLL(absolute_path_o3, winmode=0x8)

In [18]:
# Defining a function like the function in c:
def sum(value):
    sum_value = 0
    for i in range(value):
        sum_value += 1

    return sum_value

In [34]:
# Defining a value to be used
value = 2**30

In [38]:
# Generating a dataframe to compare results
columns = ["Python", "C", "Parallelized C", "C with Optimization (O3)", "Parallelized C with Optimization (O3)"]
df = pd.DataFrame(columns=columns)

In [115]:
# Start a new row
new_row = {}

### PYTHON

In [116]:
start = time.time()
sum(value)
finish = time.time()
print(f"Code executed in {finish-start} seconds")
new_row["Python"] = finish-start

Code executed in 44.22354030609131 seconds


### C

In [117]:
start = time.time()
my_func.sum(value)
finish = time.time()
print(f"Code executed in {finish-start} seconds")
new_row["C"] = finish-start

Code executed in 2.9516704082489014 seconds


### Parallelized C

In [118]:
start = time.time()
my_func.sum_parallel(value)
finish = time.time()
print(f"Code executed in {finish-start} seconds")
new_row["Parallelized C"] = finish-start

Code executed in 0.27906370162963867 seconds


### C with optimization (O3)

In [119]:
start = time.time()
my_func_o3.sum(value)
finish = time.time()
print(f"Code executed in {finish-start} seconds")
new_row["C with Optimization (O3)"] = finish-start

Code executed in 0.8196051120758057 seconds


### Parallelized C with optimization (O3)

In [120]:
start = time.time()
my_func_o3.sum_parallel(value)
finish = time.time()
print(f"Code executed in {finish-start} seconds")
new_row["Parallelized C with Optimization (O3)"] = finish-start

Code executed in 0.08601951599121094 seconds


### Adding the row to the DataFrame

In [121]:
# Adding the row to the DataFrame
df.loc[len(df)] = new_row

### Checking the Results

In [122]:
df

,Python,C,Parallelized C,C with Optimization (O3),Parallelized C with Optimization (O3)
0,43.106490,2.953668,0.281062,0.805181,0.084524
1,44.482627,2.977674,0.353080,0.832188,0.103022
2,44.353094,2.967670,0.307070,0.824187,0.103024
3,44.136875,2.955960,0.323074,0.804181,0.089020
4,44.046071,2.957192,0.282063,0.822186,0.095021
5,44.422154,2.963670,0.304068,0.803182,0.087019
6,44.039393,2.961010,0.289065,0.809186,0.083019
7,44.019779,2.957667,0.288065,0.801181,0.083019
8,44.328374,2.960408,0.323073,0.808183,0.081019
9,44.223540,2.951670,0.279064,0.819605,0.086020


In [123]:
df.mean()

Python                                   44.115840
C                                         2.960659
Parallelized C                            0.302968
C with Optimization (O3)                  0.812926
Parallelized C with Optimization (O3)     0.089471
dtype: float64

In [136]:
print(f"For this test, comparing functions of a for loop that adds 1 to the final result {value} times, here are the results:")
for idx, item in enumerate(df.mean()):
    if idx == 0:
        continue

    print(f"{df.columns[idx]} is {(df.mean().iloc[0] / item):.2f} times faster than Python")

For this test, comparing functions of a for loop that adds 1 to the final result 1073741824 times, here are the results:
C is 14.90 times faster than Python
Parallelized C is 145.61 times faster than Python
C with Optimization (O3) is 54.27 times faster than Python
Parallelized C with Optimization (O3) is 493.08 times faster than Python
